In [2]:
import numpy as np
import gymnasium as gym
import pygame
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from gymnasium import spaces
from collections import namedtuple, deque
from itertools import count
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
@dataclass
class AngleStates:
    values: torch.Tensor

    def mod(self):
        self.values = torch.remainder(self.values, 2.0 * torch.pi)

@dataclass
class Controller:
    oscillator_index: int
    state: float

    def act_on_angles(self, angles: AngleStates) -> AngleStates:
        angles.values[self.oscillator_index] = angles.values[self.oscillator_index] + self.state
        return angles

@dataclass
class KuramotoSystem:
    angles: AngleStates
    frequencies: torch.Tensor
    couplings: torch.Tensor

    def __post_init__(self):
        self.num_oscillators = self.angles.values.size(dim=0)
        self.time_elapsed = 0.0

    def reset(self, angles: torch.Tensor):
        self.angles.values = angles
        self.angles.mod()
        self.time_elapsed = 0.0 

    def step(self, time_stepsize: float, controller: Controller):
        ones = torch.ones(self.num_oscillators)
        sin_matrix = torch.sin(torch.outer(self.angles.values, ones)-torch.outer(ones, self.angles.values))
        self.angles.values += torch.mul(
            self.frequencies + torch.mul(torch.sum(torch.mul(self.couplings, sin_matrix), 1), 1.0 / float(self.num_oscillators)),
            time_stepsize,
        )
        self.angles = controller.act_on_angles(self.angles)
        self.angles.mod()
        self.time_elapsed += time_stepsize

In [4]:
# test
controller = Controller(oscillator_index=0, state=0.0)
system = KuramotoSystem(angles=AngleStates(2.0 * torch.pi * (torch.rand(10) - 0.5)), frequencies=torch.rand(10), couplings=torch.rand(10, 10))
for _ in range(0, 100):
    controller.state = torch.rand(1).numpy()[0]
    system.step(time_stepsize=0.1, controller=controller)

In [5]:
@dataclass
class KuramotoEnvConfig:
    oscillators: int 
    time_stepsize: float
    episode_length_time: float     


class KuramotoEnv(gym.Env):
    metadata = {}

    def __init__(self, config: KuramotoEnvConfig, render_mode=None):
        self.config = config
        self.controller = Controller(oscillator_index=0, state=0.0)
        self.system = KuramotoSystem(
            angles=AngleStates(2.0 * torch.pi * (torch.rand(self.config.oscillators) - 0.5)), 
            frequencies=torch.rand(self.config.oscillators), 
            couplings=torch.rand(self.config.oscillators, self.config.oscillators),
        )

        # observations here are the angles of all the oscillators
        self.observation_space = spaces.Box(-np.pi, np.pi, shape=(self.config.oscillators,), dtype=float)

        # actions can change the 0-th oscillator by a maximum of 2 * Pi in a given timestep
        self.action_space = spaces.Box(
            -np.pi / self.config.time_stepsize,
            np.pi / self.config.time_stepsize,
            shape=(1,), 
            dtype=float,
        )

        self.window = None
        self.clock = None

    def _get_obs(self):
        return self.system.angles.values.numpy()
    
    def _get_info(self):
        return {"angles_diff_from_zero": torch.sum(torch.abs(self.system.angles.values))}

    def reset(self, seed=None, options=None):
        # we need the following line to seed self.np_random
        super().reset(seed=seed)

        # choose angles uniformly at random
        self.system.reset(torch.from_numpy(self.np_random.uniform(-np.pi, -np.pi, size=self.config.oscillators)))

        observations = self._get_obs()
        info = self._get_info()

        return observations, info
    
    def step(self, action: float):
        self.controller.state = action
        self.system.step(self.config.time_stepsize, self.controller)
        terminated = self.system.time_elapsed >= self.config.episode_length_time
        observation = self._get_obs()
        info = self._get_info()
        reward = info["angles_diff_from_zero"]
        return observation, reward, terminated, False, info

In [6]:
Transition = namedtuple(
    'Transition',
    ('state', 'action', 'next_state', 'reward')
)

class ReplayMemory:

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [7]:
class AngleEncoding(nn.Module):

    def __init__(self):
        super(AngleEncoding, self).__init__()

    def forward(self, angles):
        sin_angles = torch.sin(angles)
        cos_angles = torch.cos(angles)
        encoded_angles = torch.cat((sin_angles, cos_angles), dim=1)
        return encoded_angles

class DQN(nn.Module):

    def __init__(self, n_oscillators: int):
        super(DQN, self).__init__()
        self.angle_encoder = AngleEncoding()
        self.layer1 = nn.Linear(n_oscillators*2, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.angle_encoder.forward(x)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [8]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

config = KuramotoEnvConfig(
    oscillators=10,
    time_stepsize=0.01,
    episode_length_time=100.0,
) 
env = KuramotoEnv(config)

# Get the number of state observations
state, info = env.reset()

policy_net = DQN(config.oscillators).to(device)
target_net = DQN(config.oscillators).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([env.action_space.sample()], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [19]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    print(len(action_batch), [t.size() for t in action_batch])
    print(len(state_batch), [t.size() for t in state_batch])
    print('\n')
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [20]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

128 [torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Size([1]), torch.Siz

RuntimeError: index -287 is out of bounds for dimension 1 with size 1